In [17]:
import json
import pandas as pd
import requests
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time

# 네이버 음식점 id 가져오기 (차후 따로 파일을 만들어서 id 리스트만 저장해놓고 아래 크롤링 시작 예정)

In [18]:
#크롬 웹드라이버 불러오기
driver = webdriver.Chrome()
res = driver.get('https://m.place.naver.com/place/list?query=강남역 맛집&level=top')
driver.implicitly_wait(20)

In [19]:
#2차 크롤링을 위한 bs4 셋팅
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [20]:
#body부분을 잡기 위해 쓸데없이 버튼을 클릭해줌
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[1]/a').click()

#검색결과가 모두 보이지 않기 때문에 page down을 눌러 끝까지 펼쳐준다.
for scroll in range(0,100):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)


In [21]:
from bs4 import BeautifulSoup
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
soup = bs.select_one('div.YluNG')
naver_info = soup.select('li.VLTHu')

In [22]:
from tqdm import tqdm
test = bs.find_all('a', class_='P7gyV')
test_res = []
test_str = []

for x in tqdm(test):
    str_x = str(x).split('/')[4].split('?')[0]
    print(str_x)
    
    if not str_x[0].isdigit():
        continue
    
    # Check if str_x contains any English characters
    if any(char.isalpha() for char in str_x):
        continue
    
    if str_x not in test_res:
        test_res.append(str_x)



100%|██████████| 808/808 [00:00<00:00, 11084.90it/s]

JdVDBBHny_KvXyWhCdiJh48dGdQGKMzosF-wMOJnHGfcNh5zLPDx_EWR5xjUfI2kn-ty57n5cabcRBIUo_CLtVUI59Ad5cjQ7ckAEV-u-1vtHIxkrCCpEeZkemZsEHw-0VayGFLxSW_VZLDQpkGA5Dwxuee5slXciB6QulqKOenrOMErdicL5AIlkDYe3YrIFNpJUu6OOrzaKayGToAMfpbzgsYgR0jDEU2CDuvhBwk68IRFqEP_vlxnEcX9j3KYYcV1HpuYONeitzbD00_PEbRt-ZOUHK9NJMaDq3igndYJfYdQ3waJmALWw8FLFGNWJUar1O77aaGRwMqMa8PRKA0_qVI8Loo9zQhaZug8JNE-Tepi7ZVi5wKS0fisxUn50p1W8SJc7NnkjB_SQZIhVUWf0CyUdpkxIeQcvRVTvgOQGuPRouNNY0TwTkA7jP9EROeGQxmcUYjnVrq1DxxPBY9HFnpZQYDblssV179gK24EeQG02bfFp_trINJWvwI_ZLsojle0wBh7i4_78CX5iTeusC9_euxu0itqvqqRIhaUnRBkL8pWygZ1oqcAQ-AhOsY4JSEAKzO4GPywLZl7lrVMfCeHtXp-8t-xJkePi6TryDJihs6L8TX4AvYbONTf6N7aGcYnDyJ1yN_qB1zALJ0usnlJqtNz3SXOFd-AF3wSF1uwVRPmgE8joFCB41QGuVVvqdK31a3Iu4uTtECOROc3x3_eZAHTqpGHKjSmfUuGHkPLKaX2GRGRPA19n5KaAGwPwvwZQyhOnfzSL3wsRPo7xd9WAMFcyJeGLLs6OpLM9NeOWj06eR6sbsaHjk_2pazsJ7OMCpVt48c_qe-C_0cccqdkv0hHiUitoSYA47U=
JdVDBBHny_KvXyWhCdiJh48dGdQGKMzosF-wMOJnHGfcNh5zLPDx_EWR5xjUfI2kn-ty57n5cabcRBIUo_CLtVUI59Ad5cjQ7ckAEV-u-1vtHIxkrCCpEeZkemZ

# test_res에 저장된 네이버 음식점 id로 크롤링 시작

In [73]:
test_res[1]

'37792495'

In [74]:
url = "https://api.place.naver.com/graphql"

In [75]:
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko",
    "Access-Control-Request-Headers": "content-type,x-wtm-graphql",
    "Access-Control-Request-Method": "POST",
    "Content-Length": "2590",
    "Content-Type": "application/json",
    "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZ=",
    "Origin": "https://m.place.naver.com",
    "Referer": "https://m.place.naver.com/restaurant/{test_res[i]}/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F{test_res[i]}%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F{test_res[i]}%2Freview%2Fvisitor",
    "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
}

In [76]:
all_items = []

for i in range(0,len(test_res)):
    for j in range(0,3):
        time.sleep(1)
        payload = [{
            "operationName": "getVisitorReviews",
            "variables": {
                "input": {
                    "businessId": test_res[i],
                    "businessType": "restaurant",
                    "item": "0",
                    "bookingBusinessId": "496515",
                    "page": j,
                    "size": 50,
                    "isPhotoUsed": False,
                    "includeContent": True,
                    "getUserStats": True,
                    "includeReceiptPhotos": True,
                    "cidList": ["220036", "220039", "220091", "1005863"],
                    "getReactions": True,
                    "getTrailer": True
                },
                "id": test_res[i]
            },
            "query": "query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        borderImageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        isFollowing\n        followerCount\n        followRequested\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        thumbnailRatio\n        class\n        videoId\n        videoOriginSource\n        trailerUrl\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        date\n        replyTitle\n        isReported\n        isSuspended\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      language\n      highlightOffsets\n      apolloCacheId\n      translatedText\n      businessName\n      showBookingItemName\n      bookingItemName\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      userIdno\n      loginIdno\n      receiptInfoUrl\n      reactionStat {\n        id\n        typeCount {\n          name\n          count\n          __typename\n        }\n        totalCount\n        __typename\n      }\n      hasViewerReacted {\n        id\n        reacted\n        __typename\n      }\n      nickname\n      showPaymentInfo\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    showRecommendationSort\n    itemReviewStats {\n      score\n      count\n      itemId\n      starDistribution {\n        score\n        count\n        __typename\n      }\n      __typename\n    }\n    reactionTypes {\n      name\n      emojiUrl\n      label\n      __typename\n    }\n    __typename\n  }\n}"
            }]
        headers = {
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "ko",
            "Access-Control-Request-Headers": "content-type,x-wtm-graphql",
            "Access-Control-Request-Method": "POST",
            "Content-Length": "2590",
            "Content-Type": "application/json",
            "Cookie": "NNB=JWVUUI4ZSEBGC; ASID=b4410c9600000184d649900000000067; NID_AUT=ayX11ny+DBG3UPvAL9PQDltP9OYFKXanlXyU8aJnJJIa0ETJEm1AKwG/CBf6+yxf; NID_JKL=vOmqMROEMMjvJKtCySKUNLrC7I6vNR7Du1t9ppypGXs=; ba.uuid=453a86de-7a5e-41dd-89c4-4b0b3df54fbd; NaverSuggestUse=use%26unuse; NID_SES=AAABuA3Hisfy2n7rNb5w+V9LccNaf3S1FJqFqi220NwxmY3JsjJQXYjDy9HIP16VsCWPs1S7elTCctGvJCWUiDnRiaMAETeMTGHQYot42thlyfHnOPguGLNqtOKBqJn12Zy95jb6/SC7p7fmHWqtW6qxodG1DgPh77Bt8akpGlHNWp8RdksxFEuNT29/yU6L/RbIUynmdzc5tiz9aXl2qqqe7MWuWDLQ3G8DMSXSw9maLcRCohS50gwpDxXubRaVXsy5YznGWoOfW8BdJsii0RpgIidTmJxkweXezYWdTG7ZXCFqZH3DclTPhrHF8xRKdd0hJ1neE63UPqJLawBOneQaBIR4Lql+hfDHiJCL7SbM61VwgSxSVTfMYHwD98L7PKkI89ZowXstxSfLlk2QeSFDR6oXI9z63jyFtoa8PT54Q212UZISsotyv8AQbtTzCKQ9pq6PjqflISUTIh79gnklOm6meMtbcQzBM50bJxYymHFSRbR+bHEEw199x+yahmEL521mgVOokRxQsJCCx9KN9Sea3+84XPF74Hlqb31TRvi8yaReaD3CVvgqo9A1/YpQjX1Qb6CEY/I6AGGhZjggfZ=",
            "Origin": "https://m.place.naver.com",
            "Referer": f"https://m.place.naver.com/restaurant/{test_res[i]}/review/visitor?a=rrv.more&r=&i=&m=2&ssc=Mstore.restaurant&p=3&lcsurl=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F{test_res[i]}%2Freview%2Fvisitor&px=456&py=11097&sx=456&sy=487&ua_brand_0=Not%20A(Brand&ua_version_0=99&ua_brand_1=Google%20Chrome&ua_version_1=121&ua_brand_2=Chromium&ua_version_2=121&ua_architecture=x86&ua_platform=Windows&ua_platformVersion=15.0.0&ua_uaFullVersion=121.0.6167.140&ua_full_brand_0=Not%20A(Brand&ua_full_version_0=99.0.0.0&ua_full_brand_1=Google%20Chrome&ua_full_version_1=121.0.6167.140&ua_full_brand_2=Chromium&ua_full_version_2=121.0.6167.140&u=https%3A%2F%2Fm.place.naver.com%2Frestaurant%2F{test_res[i]}%2Freview%2Fvisitor",
            "Sec-Ch-Ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": "\"Windows\"",
            "Sec-Fetch-Dest": "empty",
            "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Site": "same-site",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
            "X-Wtm-Graphql": "eyJhcmciOiIxMTA5MDI0NzA5IiwidHlwZSI6InJlc3RhdXJhbnQiLCJzb3VyY2UiOiJwbGFjZSJ9"
        }
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        a = response.json()
        if(a[0]['data']['visitorReviews']!=None):
            all_items.extend(a[0]['data']['visitorReviews']['items'])
# 모든 items를 DataFrame으로 변환
df = pd.DataFrame(all_items)
df['businessName']
# DataFrame을 CSV 파일로 저장
df.to_csv('visitor_reviews.csv', index=False, encoding='utf-8')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [72]:
df[df['businessName'] == '낙원타코 강남역점']['body']

800    주말에 점심 식사때 갔다가 커플세트가 있긴 했지만 단품메뉴로 잘 먹고 왔어요. 낙원...
801    90년대 아메리칸 패밀리 레스토랑 분위기. 맛있다. 직원들이 많아 테이블을 주시하고...
802    낙원파히타랑 치킨 퀘사디아 먹었어요.\n맛있고 2인이었고 양도 적당해요.\n매장이 ...
803    ᴍ ᴇ ɴ ᴜ 낙원파히타(42900) + 치킨퀘사디아(17900) + 과카몰리(49...
804    연말이라 그런지 웨이팅이 꽤 있었지만 회전율이 좋아서 금방 착석할 수 있었어요!😆 ...
                             ...                        
995                                                 맛있어여
996                                              낙원타코 최공
997                                                  좋아요
998                              맛있는데 지점 옮기기 전이 더 맛있었어요.
999    맛있게 잘 먹고 왔습니다~ 요기 다녀오고 나서부터 멕시코 음식 찾아먹기 시작했어요^...
Name: body, Length: 200, dtype: object